In [3]:
import numpy as np
import pandas as pd
import time
from sklearn import metrics
from sklearn import preprocessing
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn.metrics import confusion_matrix
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
#######################################
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier

In [4]:
def build_Dataframes():
    t1 = pd.read_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/IDS2018/Data_O/F_18norm50_tr1')
    df1 = pd.DataFrame(t1)
    t2 = pd.read_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/IDS2018/Data_O/F_18norm50_te1')
    df2 = pd.DataFrame(t2)
    
    res = dict(); 
    res['df1']=df1
    res['df2']=df2
    
    return res
    

In [5]:
 def lv1_Replace(df,index):
    df[index][df[index]!='Normal']='Attack'   
  #  df[index] = df[index].replace({'DoS': 'Attack','Probe': 'Attack', 'R2L': 'Attack', 'U2R': 'Attack' })
    return df   

In [6]:
 def lv2_Replace(df,index):
    df[index] = df[index].replace({'DoS':'G1', 'DDoS':'G1', 'BruteForce':'G2', 'Web':'G2', 'Infilteration':'G2', 'Bot':'G1'})
    return df   

In [7]:
def train_test_data(df_train,df_test,index):
 
    df_data=df_train.iloc[:,0:index]
    df_target=df_train.iloc[:,index]

    df_data1=df_test.iloc[:,0:index]
    df_target1=df_test.iloc[:,index]

    traindata1 = np.array(df_data)
    trainlabel1 = np.array(df_target)

    testdata1 = np.array(df_data1)
    testlabel1 = np.array(df_target1)
    
    res = dict(); 
    res['traindata']=traindata1
    res['trainlabel']=trainlabel1
    res['testdata']=testdata1
    res['testlabel']=testlabel1
    
    return res

In [8]:
def bin_Accuracy(expected,predicted):
    accuracy = accuracy_score(expected, predicted)
    recall = recall_score(expected, predicted, average='micro')
    precision = precision_score(expected, predicted , average='micro')
    f1 = f1_score(expected, predicted , average='micro')
    print("Accuracy")
    print("%.3f" %accuracy)
    print("precision")
    print("%.3f" %precision)
    print("recall")
    print("%.3f" %recall)
    print("f-score")
    print("%.3f" %f1)
    #     cm = metrics.confusion_matrix(expected, predicted)
    #     print(cm)
    #     tpr = float(cm[0][0])/np.sum(cm[0])
    #     fpr = float(cm[1][1])/np.sum(cm[1])

    #     print("fpr")
    #     print("%.3f" %fpr)
    #     print("tpr")
    #     print("%.3f" %tpr)
    #     print(predicted)

In [28]:
def multi_Accuracy(expected,predicted,gflag):
        
        if(gflag==1):
            Labels=att_Grp1()
        else:
            Labels=att_Grp2()
        
        cm=metrics.multilabel_confusion_matrix(expected, predicted,labels=Labels)
        
        tn = cm[:, 0, 0]
        tp = cm[:, 1, 1]
        fn = cm[:, 1, 0]
        fp = cm[:, 0, 1]
        acc=(tp+tn)/(tp+tn+fp+fn)
        prec=tp/(tp+fp)
        rec=tp/(tp+fn)
        f1= (2*tp)/(2*tp+fp+fn)
        print(Labels)
        df_res=pd.DataFrame()
        df_res['Accuracy']=acc
        df_res['Precision']=prec
        df_res['Recall']=rec
        df_res['Score']=f1
        df_res=df_res.round(5)
        if(gflag==1):
            df_res.to_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/Res_files/copy_paste_res1.csv',index=False)
        else:
            df_res.to_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/Res_files/copy_paste_res2.csv',index=False)
        
        print("truepositive")
        print(tp)
        sum1=sum(tp)
        print("SUM TP")
        print(sum1)
        
        print("falsenegative")
        print(fn)
        sum2=sum(fn)
        print("SUM FN")
        print(sum2)
        
        print("Accuracy")
        print(acc)
        print("Precision")
        print(prec)
        print("Recall")
        print(rec)
        print("F1_score")
        print(f1)
    

In [27]:
def model_Accuracy(model,tra_data,tes_data,tra_label,tes_label,bflag,gflag):
    t0=time.time()
    model.fit(tra_data, tra_label)
    print("training time:", round(time.time()-t0, 3), "s")
    print(model)
    expected = tes_label
    
    t1=time.time()
    predicted = model.predict(tes_data)
    print("Testing time:", round(time.time()-t1, 3), "s")
    
    
    if(bflag==1):
        bin_Accuracy(expected,predicted)
    else:
        multi_Accuracy(expected,predicted,gflag)
    
    return predicted


In [11]:
def lv2_Preprocessing(dframe1,pre1,index):
    #TestDataSet Preprocessing
    df1=dframe1['df2'].copy(deep=True)
    df1['target']=pre1
    df_nor=df1[df1['target']=='Normal']
    df_nor.to_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/Comb_files/Att_Norm1.csv',index=False)
    df_att=df1[df1['target']=='Attack']
    df_att_nor=df_att[df_att[index]=='Normal']
    df_att=df_att[df_att[index]!='Normal']
    df_att_lv3=df_att.copy(deep=True)
    df_att=lv2_Replace(df_att,index)
    df_att_nor.to_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/Comb_files/Nor_Att1.csv',index=False)
    del df_att['target']
    
    #TrainDataSet Preprocessing
    df_l2_tra=dframe1['df1'].copy(deep=True)
    df_l2_tra=df_l2_tra[df_l2_tra[index]!='Normal']
    df_l2_tra=lv2_Replace(df_l2_tra,index)      
    
    #Store the results and return
    res=dict()
    
    res['df_att_lv3']=df_att_lv3
    res['df_l2_tes']=df_att
    res['df_l2_tra']=df_l2_tra
    
    
    return res

In [12]:
def att_Grp1():
    list1=['DoS','DDoS','Bot']
    return list1

In [13]:
def att_Grp2():
    list2=[ 'BruteForce','Infilteration','Web']
    return list2

In [14]:
def lv3_Preprocessing(dframe1,dframe2,pre2,index):
    #Testdataset
    df1=dframe2['df_att_lv3'].copy(deep=True)
    df1['target1']=pre2
    df_g1=df1[df1['target1']=='G1']
    df_g2=df1[df1['target1']=='G2']
    del df_g1['target1']
    del df_g2['target1']
    
    l1=att_Grp1()
    l2=att_Grp2()
    
    
    df_g1_g2=df_g1[df_g1[index].isin(l2)]
   # df_g1=df_g1[df_g1[index].isin(l1)]
    
    df_g2_g1=df_g2[df_g2[index].isin(l1)]
   # df_g2=df_g2[df_g2[index].isin(l2)]
    
    df_g1_g2.to_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/Comb_files/g1_g2.csv',index=False)
    df_g2_g1.to_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/Comb_files/g2_g1.csv',index=False)
    
    #TrainDataSet Preprocessing
    df_l3_tra=dframe1['df1'].copy(deep=True)
    df_l3_tra=df_l3_tra[df_l3_tra[index]!='Normal']
    df_l3_tra_g1=df_l3_tra[df_l3_tra[index].isin(l1)]
    df_l3_tra_g2=df_l3_tra[df_l3_tra[index].isin(l2)]
    
    #Store the results and return
    res=dict() 
    
    res['df_l3_g1_tes']=df_g1
    res['df_l3_g1_tra']=df_l3_tra_g1
    
    res['df_l3_g2_tes']=df_g2
    res['df_l3_g2_tra']=df_l3_tra_g2
    
    
    return res
    
    

In [15]:
def total_Accuracy(dframe3,pre3,pre4,index):
    d1 = pd.read_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/Comb_files/Att_Norm1.csv')
    df_att_nor1 = pd.DataFrame(d1)
    d2 = pd.read_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/Comb_files/Nor_Att1.csv')
    df_nor_att1 = pd.DataFrame(d2)
    d3 = pd.read_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/Comb_files/g2_g1.csv')
    dff_g2_g1 = pd.DataFrame(d3)
    d4 = pd.read_csv('/Users/srivatsan/Sri/M.Tech_project/Multilevel_hierarchy_class/Comb_files/g1_g2.csv')
    dff_g1_g2 = pd.DataFrame(d4)
    d5_g1=dframe3['df_l3_g1_tes']
    d5_g1['target2']=pre3
    del d5_g1['target']
    d6_g2=dframe3['df_l3_g2_tes']
    d6_g2['target2']=pre4
    del d6_g2['target']
    
    list_a=list(df_att_nor1[index])
    list_b=list(df_nor_att1[index])
    list_a=list_a+list_b
    list_b=list(dff_g2_g1[index])
    list_a+=list_b
    list_b=list(dff_g1_g2[index])
    list_a+=list_b
    list_b=list(d5_g1[index])
    list_a+=list_b
    list_b=list(d6_g2[index])
    list_a+=list_b
   
    list_a1=list(df_att_nor1['target'])
    list_b=list(df_nor_att1['target'])
    list_a1=list_a1+list_b
    list_b=list(dff_g2_g1['target'])
    list_a1+=list_b
    list_b=list(dff_g1_g2['target'])
    list_a1+=list_b
    list_b=list(d5_g1['target2'])
    list_a1+=list_b
    list_b=list(d6_g2['target2'])
    list_a1+=list_b
    
    df_res1=pd.DataFrame()
    df_res1['Exp']=list_a
    df_res1['Pre']=list_a1
    
    exp1=np.array(df_res1['Exp'])
    pre1=np.array(df_res1['Pre'])
    
    
    accuracy = accuracy_score(exp1, pre1)
    recall = recall_score(exp1, pre1, average='micro')
    precision = precision_score(exp1, pre1 , average='micro')
    f1 = f1_score(exp1, pre1 , average='micro')
    
    print("#########")
    print("Total Hierarchical F1_score")
    print(round(f1,5))
    

    

In [16]:
def MH_Model2(model2,dframe1,index1,index_str):
    dframe2=lv2_Preprocessing(dframe1,dframe1['pre1'],index_str)
    df3 = dframe2['df_l2_tra'].copy(deep=True)
    df4 = dframe2['df_l2_tes'].copy(deep=True)
    ttd2=train_test_data(df3,df4,index1)
    print("Level_2 Accuracy")
    pre2=model_Accuracy(model2,ttd2['traindata'],ttd2['testdata'],ttd2['trainlabel'],ttd2['testlabel'],1,0)
    dframe2['pre2']=pre2
    
    print("################")
     
    return dframe2
   


In [17]:
def MH_Model3(model_3_1,model_3_2,dframe1,dframe2,index1,index_str):
    dframe3=lv3_Preprocessing(dframe1,dframe2,dframe2['pre2'],index_str)
    df5 = dframe3['df_l3_g1_tra'].copy(deep=True)
    df6 = dframe3['df_l3_g1_tes'].copy(deep=True)
    ttd3=train_test_data(df5,df6,index1)
    
    print("Level_3_G1 Accuracy")
    pre3=model_Accuracy(model_3_1,ttd3['traindata'],ttd3['testdata'],ttd3['trainlabel'],ttd3['testlabel'],0,1)
    
    print("################")
    
    df7 = dframe3['df_l3_g2_tra'].copy(deep=True)
    df8 = dframe3['df_l3_g2_tes'].copy(deep=True)
    ttd3=train_test_data(df7,df8,index1)
    
    print("Level_3_G2 Accuracy")
    pre4=model_Accuracy(model_3_2,ttd3['traindata'],ttd3['testdata'],ttd3['trainlabel'],ttd3['testlabel'],0,2)
    
    total_Accuracy(dframe3,pre3,pre4,index_str)

In [18]:
def MH_Model1(model1,index1,index_str):
    time1=time.time()
    dframe1=build_Dataframes()
    
    df1 = dframe1['df1'].copy(deep=True)
    df2 = dframe1['df2'].copy(deep=True)
    df1=lv1_Replace(df1,index_str)
    df2=lv1_Replace(df2,index_str)
    ttd1=train_test_data(df1,df2,index1)
    print("Level_1 Accuracy")
    pre1=model_Accuracy(model1,ttd1['traindata'],ttd1['testdata'],ttd1['trainlabel'],ttd1['testlabel'],1,0)
    dframe1['pre1']=pre1
    print("################")
    
    return dframe1
    
    

In [19]:
model1 = KNeighborsClassifier()
model2 = DecisionTreeClassifier()
model3 = RandomForestClassifier()

model5 = MLPClassifier(hidden_layer_sizes=(100,100,50), max_iter=100,activation = 'relu',solver='adam',random_state=1)
model6 = LogisticRegression(tol=0.1,solver='lbfgs',multi_class='multinomial')
model7 = AdaBoostClassifier(n_estimators=100, random_state=1)

   

In [20]:
def MH_AllModel(model1,model2,model3,index1,index2):
    dframe_1=MH_Model1(model1,index1,index2)
    dframe_2=MH_Model2(model2,dframe_1,index1,index2)
    MH_Model3(model3,dframe_1,dframe_2,index1,index_str)
  

In [29]:
dframe_1=MH_Model1(model2,78,'78')
  

/Users/srivatsan/Sri/Anacond_Python/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Level_1 Accuracy
training time: 27.753 s
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
Testing time: 0.135 s
Accuracy
1.000
precision
1.000
recall
1.000
f-score
1.000
################


In [30]:
dframe_2=MH_Model2(model3,dframe_1,78,'78')


Level_2 Accuracy
training time: 15.449 s
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
Testing time: 0.178 s
Accuracy
1.000
precision
1.000
recall
1.000
f-score
1.000
################


In [32]:
MH_Model3(model2,model1,dframe_1,dframe_2,78,'78')


Level_3_G1 Accuracy
training time: 2.45 s
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')
Testing time: 0.029 s
['DoS', 'DDoS', 'Bot']
truepositive
[40000 39987 39996]
SUM TP
119983
falsenegative
[0 0 0]
SUM FN
0
Accuracy
[1.         0.99980834 1.        ]
Precision
[1.         0.99942514 1.        ]
Recall
[1. 1. 1.]
F1_score
[1.         0.99971249 1.        ]
################
Level_3_G2 Accuracy
training time: 121.152 s
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')
Testing time: 172.334 s
['Brut